In [48]:
import os
import config
import langchain.llms
import google.generativeai as genai


In [49]:
from langchain_google_genai import GoogleGenerativeAI

ModuleNotFoundError: No module named 'langchain_google_genai'

In [3]:
api_key = os.environ.get('API_KEY')

In [42]:
genai.configure(api_key=api_key)
model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("Explain how AI works")
print(response.text)

AI, or Artificial Intelligence, doesn't work in a single, unified way. Instead, it encompasses a vast range of techniques and approaches, all aiming to mimic human intelligence in machines.  The core idea is to create systems that can learn, reason, and act autonomously. Here's a breakdown of some key aspects:

**1. Data is King:**  Most AI systems rely heavily on data.  They learn patterns and relationships from vast amounts of data, enabling them to make predictions, classifications, or decisions.  The quality and quantity of this data significantly impact the performance of the AI.

**2. Algorithms are the Engine:**  Algorithms are sets of rules and instructions that tell the AI how to process and learn from the data. Different algorithms are suited for different tasks.  Some common types include:

* **Machine Learning (ML):** This is a subset of AI where systems learn from data without explicit programming.  Instead of being explicitly told what to do, they identify patterns and re

## creating an SQL database object

In [18]:
from langchain.utilities import SQLDatabase

db_user = "root"
db_password = "root123"
db_host = "localhost"
db_name = "atliq_tshirts"

try:
    # Attempt to create a database connection
    db = SQLDatabase.from_uri(f"mysql+pymysql://{db_user}:{db_password}@{db_host}/{db_name}", sample_rows_in_table_info=3)
    print("DATABASE CONNECTED SUCCESSFULLY")
except Exception as e:
    # Print failure message and the specific error
    print("DATABASE CONNECTION FAILED")
    print(f"Error: {str(e)}")
    exit(1)

DATABASE CONNECTED SUCCESSFULLY


### creating an SQL chain

In [41]:
from langchain_experimental.sql import SQLDatabaseChain

In [44]:
db_chain = SQLDatabaseChain.from_llm(model, db, verbose=True)
qn = db_chain("How many tshirts are there in the database?")

ValidationError: 2 validation errors for LLMChain
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of
llm.is-instance[Runnable]
  Input should be an instance of Runnable [type=is_instance_of, input_value=genai.GenerativeModel(
  ...   cached_content=None
), input_type=GenerativeModel]
    For further information visit https://errors.pydantic.dev/2.10/v/is_instance_of